In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, AutoTokenizer, AutoModelForCausalLM
import torch
import os
from dotenv import load_dotenv
import sounddevice as sd
import numpy as np
import tkinter as tk
from threading import Thread
# Load environment variables and Hugging Face token
load_dotenv()
huggingface_token = os.getenv('HUGGINGFACE_TOKEN')


text_gen_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it", token=huggingface_token)
text_gen_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", token=huggingface_token)


device_text = "mps" if torch.backends.mps.is_available() else "cpu"
text_gen_model = text_gen_model.to(device_text)

# Initialize the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
print(text_gen_model)

In [ ]:


# Global variable to control the recording state
is_recording = False
audio_data = np.array([])


def record_audio(fs=16000):
    """Continuously record audio until stopped."""
    global is_recording, audio_data
    with sd.InputStream(samplerate=fs, channels=1, callback=callback):
        while is_recording:
            sd.sleep(100)  # Small sleep to avoid locking up the CPU


def callback(indata, frames, time, status):
    """This is called for each audio block from the microphone."""
    global audio_data
    audio_data = np.append(audio_data, indata.copy())


def toggle_recording():
    """Toggle the recording state between start and stop."""
    global is_recording, audio_data
    if not is_recording:
        # Start recording
        is_recording = True
        audio_data = np.array([])
        Thread(target=record_audio).start()  # Start recording in a background thread
    else:
        # Stop recording
        is_recording = False
        process_audio(audio_data)  # Process the recorded audio


def process_audio(audio_data, fs=16000):
    """Process the recorded audio and update the transcription label."""
    # Preprocess the audio to match the model's expected format
    input_values = processor(
        audio_data, return_tensors="pt", sampling_rate=fs
    ).input_values

    # Move to the same device as the model
    input_values = input_values.to(model.device)

    # Perform inference
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode the predicted ids
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Update the transcription label text in the main thread
    def update_transcription_label():
        transcription_label.config(text=f"Transcribed Text: {transcription}")
        generate_text(transcription)

    root.after(0, update_transcription_label)


def generate_text(transcription):
    print("Starting text generation...")

    def background_generate():
        print("Starting text generation inside...")
        input_ids = text_gen_tokenizer.encode(transcription, return_tensors="pt").to(device)
        
        try:
            # Generate text using the model and tokenizer
            output_sequences = text_gen_model.generate(input_ids, max_length=50, num_return_sequences=1)
            generated_text = text_gen_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
            print("Generated text:", generated_text)  # For debugging
        except Exception as e:
            print("An error occurred:", e)
            generated_text = "Error in generating text."

        # Function to update the GUI with the generated text
        def update_generated_text_label():
            generated_text_label.config(text=f"Model Response: {generated_text}")

        # Schedule the GUI update to run in the main thread
        root.after(0, update_generated_text_label)

    # Run the text generation in a background thread to avoid blocking the GUI
    Thread(target=background_generate).start()


# Set up the GUI
root = tk.Tk()
root.title("Voice Recorder")

# Add a record button
record_button = tk.Button(root, text="Start Recording", command=toggle_recording)
record_button.pack(pady=20)

# Add a label widget for displaying the transcription
transcription_label = tk.Label(
    root, text="Transcription will appear here...", wraplength=400
)
transcription_label.pack(pady=10)

generated_text_label = tk.Label(
    root, text="Generated text will appear here...", wraplength=400
)
generated_text_label.pack(pady=10)


def update_button_text():
    """Update the button text based on the recording state."""
    if is_recording:
        record_button.config(text="Stop Recording")
    else:
        record_button.config(text="Start Recording")
    root.after(100, update_button_text)


root.after(100, update_button_text)  # Check every 100ms to update the button text
root.mainloop()